In [60]:
import pandas as pd
import numpy as np

# import du csv
data = pd.read_csv('csv_correction.csv')
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,fk_pied,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37.0,EN PLACE,Jeune,semi libre,gazon,Alignement,Non,15.0,5.0,0.0,QUERUB,VILLE,Feuillu,Non
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,10.0,0.0,PINNIGnig,VILLE,Conifère,Non
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116.0,REMPLACÉ,Adulte,semi libre,gazon,Alignement,Non,30.0,10.0,0.0,ACEPSE,VILLE,Feuillu,Non
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150.0,EN PLACE,Adulte,semi libre,gazon,Groupe,Non,50.0,2.0,0.0,ACEPLA,VILLE,Feuillu,Non
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,gazon,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,ALBJUL,VILLE,Feuillu,Non
7405,3.283436,49.848682,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,3.0,1.0,0.0,ALBJULcho,VILLE,Feuillu,Non
7406,3.297078,49.860413,Quartier de l'Europe,rue Hertz,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Isolé,Non,5.0,1.0,0.0,CEDLIB,VILLE,Conifère,Non
7407,3.291685,49.848009,Quartier du Centre-Ville,Square rue Fréreuse,3.0,2.0,15.0,EN PLACE,Jeune,Libre,gazon,Alignement,Non,5.0,1.0,0.0,FAGSYLdawgol,VILLE,Feuillu,Non


In [61]:
data.columns

Index(['longitude', 'latitude', 'clc_quartier', 'clc_secteur', 'haut_tot',
       'haut_tronc', 'tronc_diam', 'fk_arb_etat', 'fk_stadedev', 'fk_port',
       'fk_pied', 'fk_situation', 'fk_revetement', 'age_estim',
       'fk_prec_estim', 'clc_nbr_diag', 'fk_nomtech', 'villeca', 'feuillage',
       'remarquable'],
      dtype='object')

In [62]:
from sklearn import preprocessing, feature_selection, model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import cross_val_predict, cross_val_score
import time
from imblearn.over_sampling import SMOTE


In [63]:
indexDrop = data[((data['fk_arb_etat'] == 'ABATTU') | (data['fk_arb_etat'] == 'EN PLACE') | (data['fk_arb_etat'] =='REMPLACÉ') | (data['fk_arb_etat'] == 'SUPPRIMÉ') )].index
indexDrop

Index([   0,    1,    2,    3,    5,    6,    7,    8,    9,   10,
       ...
       7399, 7400, 7401, 7402, 7403, 7404, 7405, 7406, 7407, 7408],
      dtype='int64', length=7217)

In [64]:
data.drop(indexDrop, inplace = True)

In [65]:
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,fk_pied,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,gazon,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non
73,3.270746,49.844440,Quartier Saint-Martin - Oëstres,Rue Jean Mermoz,12.0,3.0,160.0,Essouché,Adulte,réduit relâché,gazon,Isolé,Non,50.0,10.0,1.0,PLAACE,VILLE,Feuillu,Non
76,3.270951,49.844588,Quartier Saint-Martin - Oëstres,Rue Jean Mermoz,13.0,4.0,155.0,Essouché,Adulte,réduit relâché,gazon,Isolé,Non,50.0,10.0,2.0,PLAACE,VILLE,Feuillu,Non
92,3.294703,49.849927,Quartier Remicourt,Parc des Champs-Elysées,24.0,9.0,245.0,Essouché,Adulte,semi libre,gazon,Groupe,Non,80.0,20.0,2.0,ACEPLA,VILLE,Feuillu,Non
104,3.294280,49.849615,Quartier Remicourt,Parc des Champs-Elysées,20.0,14.0,180.0,Essouché,Adulte,semi libre,gazon,Groupe,Non,80.0,20.0,2.0,TILCOR,VILLE,Feuillu,Non
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6886,3.296399,49.848276,Quartier Remicourt,Place du Nain d'Alsace,9.0,3.0,85.0,Essouché,Jeune,architecturé,Terre,Isolé,Non,25.0,5.0,0.0,BETPEN,VILLE,Feuillu,Non
6887,3.296366,49.848250,Quartier Remicourt,Place du Nain d'Alsace,9.0,3.0,86.0,Essouché,Jeune,architecturé,Terre,Isolé,Non,25.0,5.0,0.0,BETPEN,VILLE,Feuillu,Non
6888,3.296433,49.848229,Quartier Remicourt,Place du Nain d'Alsace,9.0,4.0,98.0,Essouché,Jeune,architecturé,Terre,Isolé,Non,25.0,5.0,0.0,BETPEN,VILLE,Feuillu,Non
6977,3.265997,49.836176,Quartier Saint-Martin - Oëstres,Ecole maternelle Ernest Lavisse,5.0,2.0,25.0,Non essouché,Jeune,Libre,Terre,Isolé,Non,10.0,2.0,0.0,FAGSYLpur,VILLE,Feuillu,Non


In [66]:
data["fk_arb_etat"].value_counts()

fk_arb_etat
Essouché        165
Non essouché     27
Name: count, dtype: int64

passage de donnees str en int

In [67]:
# 'fk_arb_etat_bool',
list_col = ['fk_stadedev', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']
for col in list_col:
  encoder = preprocessing.LabelEncoder()
  fk_stadedev_encoded = encoder.fit_transform(data[col])
  #print(encoder.categories_)
  data[col] = fk_stadedev_encoded

In [68]:
all_cols_list = ["haut_tot", 'haut_tronc', 'tronc_diam', 'fk_stadedev', 'age_estim', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']

In [69]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data[all_cols_list], data["fk_arb_etat"], test_size=0.2, random_state=42, stratify=data["fk_arb_etat"])

## SMOTE

In [70]:
sm = SMOTE(random_state=42)
X_smoted, y_smoted = sm.fit_resample(X_train, y_train)

## grid search

In [71]:
# train
classifier = RandomForestClassifier(random_state=42, n_jobs=-1)
params = {"n_estimators": [50, 100, 150, 200, 250, 300]}
gscv = GridSearchCV(estimator=classifier, param_grid=params, scoring="accuracy", cv=5, n_jobs=-1)
classifier_with_grid_search = gscv.fit(X_train, y_train)  # le meilleur modele

print(classifier_with_grid_search.best_params_)
from sklearn.metrics import accuracy_score
cvs = cross_val_predict(gscv, X_smoted, y_smoted, cv = 3, n_jobs=-1)

best_n = classifier_with_grid_search.best_params_["n_estimators"]
best_n


{'n_estimators': 150}


150

### validation

In [72]:
confusion_matrix(y_smoted, cvs, normalize="true")

array([[0.87022901, 0.12977099],
       [0.06870229, 0.93129771]])

### tests

In [73]:
from sklearn.metrics import accuracy_score

y_test_predicted = classifier_with_grid_search.predict(X_test)
accuracy_score(y_test, y_test_predicted)

0.9230769230769231

In [74]:
y_pred = classifier_with_grid_search.predict(X_test)
confusion_matrix(y_test, y_pred, normalize="true")

array([[1. , 0. ],
       [0.6, 0.4]])

In [75]:
y_pred = classifier_with_grid_search.predict(X_test)
y_proba = classifier_with_grid_search.predict_proba(X_test)

# Afficher la matrice de confusion et le rapport de classification
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[34  0]
 [ 3  2]]
              precision    recall  f1-score   support

    Essouché       0.92      1.00      0.96        34
Non essouché       1.00      0.40      0.57         5

    accuracy                           0.92        39
   macro avg       0.96      0.70      0.76        39
weighted avg       0.93      0.92      0.91        39



In [76]:
results = X_test.copy()
results['Actual'] = y_test
results['Predicted'] = y_pred
results['Probability_False'] = y_proba[:, 0]
results['Probability_True'] = y_proba[:, 1]

In [78]:
print(results.head(20))

      haut_tot  haut_tronc  tronc_diam  fk_stadedev  age_estim  clc_quartier   
227       19.0         8.0       160.0            2       80.0             2  \
680       27.0        12.0       310.0            0      100.0             6   
4342      16.0         3.0       145.0            0       60.0             6   
1576      10.0         2.0       150.0            0       50.0             4   
3973      12.0         4.0       120.0            0       40.0             6   
5045      15.0         9.0        88.0            0       30.0             7   
1387      13.0         3.0       202.0            0       80.0             3   
4981      24.0         5.0       140.0            0       30.0             1   
7078      18.0         5.0        57.0            0       35.0             1   
3355      12.0         5.0       100.0            0       30.0             6   
6241      14.0         0.0       138.0            0       30.0             6   
1766       8.0         2.0       110.0  